# MedBot

In [2]:
%%capture
!pip install unsloth vllm
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install unsloth-zoo
!rm -rf unsloth_compiled_cache

### Import

In [3]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-23 14:43:36.139336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748011416.330707      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748011416.386934      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-23 14:43:55 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-23 14:43:56 [__init__.py:239] Automatically detected platform cuda.


### Login

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")
secret_value_1 = user_secrets.get_secret("wandb")

# Use the secret token to authenticate
login(secret_value_0) #huggingface
wandb.login(key=secret_value_1) #wandb

### Load Model

In [5]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = secret_value_0
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

### Prompt

In [6]:
prompt_style = """
Below is a task description, followed by additional context and a specific query. Your objective is to deliver a well-reasoned and accurate response that thoroughly addresses the request.
Before formulating your answer, carefully analyze the question and apply a logical, step-by-step reasoning approach. Ensure your response reflects deep clinical expertise and adheres to best practices in medical diagnostics and treatment.

### Task:
You are a medical expert with specialization in clinical reasoning, diagnostics, and treatment planning. Provide a professional and evidence-based response to the medical question below.

### Question:
{}

### Response:
<think>{}"""

### Inference on Base Model

In [7]:
# Define a test question
question = """A 23-year-old male has been experiencing excessive sweating from his palms and feet for the past 10 years. 
              What is the most likely cause of this symptom?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])



<think>
Okay, so I need to figure out why a 23-year-old guy is sweating so much from his palms and feet. He's been dealing with this for a decade. Hmm, let's think about possible causes. First, I remember that excessive sweating, or hyperhidrosis, can be either focal or generalized. Focal tends to affect specific areas like the palms and feet, while generalized affects more widespread areas.

I recall that focal hyperhidrosis is often linked to nervous system issues. Maybe it's something like hyperhidrosis itself, which is a common condition. But wait, it's been 10 years. Does that mean it's just a chronic condition, or could it be related to something else?

I should also think about other possibilities. Maybe it's a side effect of another health issue. For example, diabetes can sometimes cause sweating, but I'm not sure if that's the primary issue here. Or maybe it's related to a hormonal imbalance, though that's less common.

Another angle is to consider medications. If he's on any

In [ ]:
# Define a example2 test question
question = """What is the most likely cause of restlessness and sweating in a patient being administered intravenous quinine?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Response:")[1])



<think>
Okay, so I'm trying to figure out why a patient is feeling restless and sweating while on intravenous quinine. First, I know quinine is usually used to treat malaria, but I'm not too familiar with all its side effects. Let me think through this step by step.

I remember that quinine can have several adverse effects. One of the common ones is related to the nervous system. I think it can cause things like headache, dizziness, and sometimes more serious issues like seizures or confusion. But does it cause restlessness? Maybe that's part of the central nervous system stimulation.

Also, sweating is another symptom. I'm trying to recall if quinine affects the body's temperature regulation. I think some antimalarials can cause sweating, possibly through affecting the hypothalamus or through increased metabolic rate.

Wait, I should consider the mechanism of action. Quinine works by inhibiting the red blood cell multiplication in the parasite that causes malaria. But how does that r